In [1]:
from stsci.tools import capable
capable.OF_GRAPHICS = False
from pyraf import iraf
    

In [2]:
himage = 'm30.fits'

# Gaia 

From ds9 initial catalog of gaia stars in j2000. Then create a list of them to pass to ccfind. Ccfind will output a list of the ra and dec and the pixel of the star. this will help me identify the stars i want to pass to ccmap

In [3]:
ra=[]
dec =[]
#catalog of gaia from ds9
with open('gaia2catj2000.tsv','r') as file:
    next(file)
    for line in file:
        ra.append(float(line.split()[0]))
        dec.append(float(line.split()[1]))

#Cat to use for ccfind
catds9 = 'gaiaallradecj2000.txt'

with open(catds9,'w') as file:
    for i,j in zip(ra,dec):
        file.write("{} {}\n".format(i,j))
    

In [4]:
outputnameccfind = 'gaiamatchedfindj2000'


iraf.ccfind(catds9,outputnameccfind,himage,usewcs=True,lngcol=1,latcol=2,lngunits='degrees',latunits='degrees'
            ,insystem='j2000',refsystem='j2000',projection='tan',sbox=11,cbox=5)


Input File: gaiaallradecj2000.txt  Output File: gaiamatchedfindj2000
    Image: m30.fits  Wcs: 
Insystem: j2000  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Refsystem: m30.fits logical  Projection: TAN  Ra/Dec axes: 1/2
    Coordinates: equatorial FK5 Equinox: J2000.000
    Epoch: J2014.43511528 MJD: 56816.92586


Skipping object 1 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 2 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 3 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 4 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 5 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 6 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 7 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 8 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 9 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 10 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 11 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 12 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 13 in fil

Located 4524 objects in image m30.fits



Now create a region file from the output of ccfind to plot on image and compare to gaia. I had to make a copy with no empty lines from the outputnameccfind

In [5]:
#region to plot
ccfindreg = 'gaiaccfind.reg'



with open(ccfindreg,'w') as fileone:
    with open(outputnameccfind,'r') as file:
        for line in file:
            line = line.strip()
            if not line:  # line is blank
                continue
            if line.startswith("#"):  # comment line
                continue
            text='physical; circle({},{},2) # color=red\n'.format(float(line.split()[2]),float(line.split()[3]))
            fileone.write(text)
            #print(text)



![](ccfindygaia.png)

# Selected stars

In [2]:
#region to plot
reg = 'selected.reg'



with open(reg,'w') as fileone:
    with open('manualselecgaia.txt','r') as file:
        for line in file:
            line = line.strip()
            if not line:  # line is blank
                continue
            if line.startswith("#"):  # comment line
                continue
            text='physical; circle({},{},2) # color=red\n'.format(float(line.split()[2]),float(line.split()[3]))
            fileone.write(text)
            #print(text)



# ccmap

From the image above I got the ra and dec in fk5 from gaia ctalog and the physical x and y location from nthe red region. 

In [6]:

iraf.ccmap('manualselecgaia.txt', 'coord.db',results='initialccmapgaia.txt',solutions='solinitial.txt',image=himage,xcol=3,ycol=4,
           lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           interactive='no',update='yes',insystem='fk5',projection='tan',fitgeometry='rxyscale',reject='3')

Refsystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Insystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000

Coords File: manualselecgaia.txt  Image: m30.fits
    Database: coord.db  Solution: solinitial.txt
Coordinate mapping status
    XI fit ok.  ETA fit ok.
    Ra/Dec or Long/Lat fit rms: 0.0549  0.0778   (arcsec  arcsec)
Coordinate mapping parameters
    Sky projection geometry: tan
    Reference point: 325:05:33.67  -23:10:51.34  (degrees  degrees)
    Reference point: 4953.628  4905.165  (pixels  pixels)
    X and Y scale: 0.039  0.040  (arcsec/pixel  arcsec/pixel)
    X and Y axis rotation: 179.983  359.983  (degrees  degrees)
Wcs mapping status
    Ra/Dec or Long/Lat wcs rms: 0.0549  0.0778   (arcsec  arcsec)
Updating image header wcs



# Ccfind again

after having modified the image with the small calatlog of a few stars foudn manually run ccfind again. 

In [7]:

outputnameccfind = 'gaiamatchedfindj2000p2'


iraf.ccfind(catds9,outputnameccfind,himage,usewcs=True,lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           insystem='j2000',refsystem='j2000',projection='tan',sbox=11,cbox=5)


Input File: gaiaallradecj2000.txt  Output File: gaiamatchedfindj2000p2
    Image: m30.fits  Wcs: 
Insystem: j2000  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Refsystem: m30.fits logical  Projection: TAN  Ra/Dec axes: 1/2
    Coordinates: equatorial FK5 Equinox: J2000.000
    Epoch: J2014.43511528 MJD: 56816.92586


Skipping object 1 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 2 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 3 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 4 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 5 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 6 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 7 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 8 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 9 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 10 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 11 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 12 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 13 in fil

Located 4522 objects in image m30.fits



## CCMAP from ccfind outpu

In [58]:
#create a reagion filde from ccfind if i want to

#with open(ccfindreg,'w') as fileone:
    with open(outputnameccfind,'r') as file:
        for line in file:
            line = line.strip()
            if not line:  # line is blank
                continue
            if line.startswith("#"):  # comment line
                continue
            data = line.split('\t')
           # print(data)





In [8]:
iraf.ccmap(outputnameccfind, 'coord.db',results='ccmapgaiap2.txt',solutions='solp2.txt',image=himage,xcol=3,ycol=4,
           lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           interactive='no',update='yes',insystem='fk5',projection='tan',fitgeometry='rxyscale',reject='3')

Refsystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Insystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000

Coords File: gaiamatchedfindj2000p2  Image: m30.fits
    Database: coord.db  Solution: solp2.txt
Coordinate mapping status
    XI fit ok.  ETA fit ok.
    Ra/Dec or Long/Lat fit rms: 0.0434  0.0667   (arcsec  arcsec)
Coordinate mapping parameters
    Sky projection geometry: tan
    Reference point: 325:05:33.55  -23:10:46.83  (degrees  degrees)
    Reference point: 4956.092  5020.339  (pixels  pixels)
    X and Y scale: 0.039  0.040  (arcsec/pixel  arcsec/pixel)
    X and Y axis rotation: 180.005  0.005  (degrees  degrees)
Wcs mapping status
    Ra/Dec or Long/Lat wcs rms: 0.0434  0.0667   (arcsec  arcsec)
Updating image header wcs



# ccfind again p3

In [9]:

outputnameccfind = 'gaiamatchedfindj2000p3'
ccmapresult = 'ccmapgaiap3.txt'


iraf.ccfind(catds9,outputnameccfind,himage,usewcs=True,lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           insystem='j2000',refsystem='j2000',projection='tan',sbox=11,cbox=5)


iraf.ccmap(outputnameccfind, 'coord.db',results=ccmapresult,solutions='solp2.txt',image=himage,xcol=3,ycol=4,
           lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           interactive='no',update='yes',insystem='fk5',projection='tan',fitgeometry='rxyscale',reject='3')


Input File: gaiaallradecj2000.txt  Output File: gaiamatchedfindj2000p3
    Image: m30.fits  Wcs: 
Insystem: j2000  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Refsystem: m30.fits logical  Projection: TAN  Ra/Dec axes: 1/2
    Coordinates: equatorial FK5 Equinox: J2000.000
    Epoch: J2014.43511528 MJD: 56816.92586


Skipping object 1 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 2 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 3 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 4 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 5 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 6 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 7 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 8 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 9 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 10 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 11 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 12 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 13 in fil

Located 4524 objects in image m30.fits

Refsystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Insystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000

Coords File: gaiamatchedfindj2000p3  Image: m30.fits
    Database: coord.db  Solution: solp2.txt
Coordinate mapping status
    XI fit ok.  ETA fit ok.
    Ra/Dec or Long/Lat fit rms: 0.0428  0.0657   (arcsec  arcsec)
Coordinate mapping parameters
    Sky projection geometry: tan
    Reference point: 325:05:33.52  -23:10:46.91  (degrees  degrees)
    Reference point: 4956.943  5018.266  (pixels  pixels)
    X and Y scale: 0.039  0.040  (arcsec/pixel  arcsec/pixel)
    X and Y axis rotation: 180.007  0.007  (degrees  degrees)
Wcs mapping status
    Ra/Dec or Long/Lat wcs rms: 0.0428  0.0657   (arcsec  arcsec)
Updating image header wcs



# ccfind again p4

In [10]:

outputnameccfind = 'gaiamatchedfindj2000p4'
ccmapresult = 'ccmapgaiap4.txt'


iraf.ccfind(catds9,outputnameccfind,himage,usewcs=True,lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           insystem='j2000',refsystem='j2000',projection='tan',sbox=11,cbox=5)





Input File: gaiaallradecj2000.txt  Output File: gaiamatchedfindj2000p4
    Image: m30.fits  Wcs: 
Insystem: j2000  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Refsystem: m30.fits logical  Projection: TAN  Ra/Dec axes: 1/2
    Coordinates: equatorial FK5 Equinox: J2000.000
    Epoch: J2014.43511528 MJD: 56816.92586


Skipping object 1 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 2 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 3 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 4 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 5 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 6 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 7 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 8 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 9 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 10 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 11 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 12 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 13 in fil

Located 4524 objects in image m30.fits



In [12]:
iraf.ccmap(outputnameccfind, 'coord.db',results=ccmapresult,solutions='solp2.txt',image=himage,xcol=3,ycol=4,
           lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           interactive='no',update='yes',insystem='fk5',projection='tan',fitgeometry='rxyscale',reject='3')

Refsystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Insystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000

Coords File: gaiamatchedfindj2000p4  Image: m30.fits
    Database: coord.db  Solution: solp2.txt
Coordinate mapping status
    XI fit ok.  ETA fit ok.
    Ra/Dec or Long/Lat fit rms: 0.0428  0.0657   (arcsec  arcsec)
Coordinate mapping parameters
    Sky projection geometry: tan
    Reference point: 325:05:33.52  -23:10:46.91  (degrees  degrees)
    Reference point: 4956.943  5018.267  (pixels  pixels)
    X and Y scale: 0.039  0.040  (arcsec/pixel  arcsec/pixel)
    X and Y axis rotation: 180.007  0.007  (degrees  degrees)
Wcs mapping status
    Ra/Dec or Long/Lat wcs rms: 0.0428  0.0657   (arcsec  arcsec)
Updating image header wcs



# ccfind again p5

In [13]:

outputnameccfind = 'gaiamatchedfindj2000p5'
ccmapresult = 'ccmapgaiap5.txt'


iraf.ccfind(catds9,outputnameccfind,himage,usewcs=True,lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           insystem='j2000',refsystem='j2000',projection='tan',sbox=11,cbox=5)




Input File: gaiaallradecj2000.txt  Output File: gaiamatchedfindj2000p5
    Image: m30.fits  Wcs: 
Insystem: j2000  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Refsystem: m30.fits logical  Projection: TAN  Ra/Dec axes: 1/2
    Coordinates: equatorial FK5 Equinox: J2000.000
    Epoch: J2014.43511528 MJD: 56816.92586


Skipping object 1 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 2 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 3 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 4 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 5 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 6 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 7 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 8 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 9 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 10 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 11 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 12 in file gaiaallradecj2000.txt: cannot center in image m30.fits
Skipping object 13 in fil

Located 4524 objects in image m30.fits



In [14]:

iraf.ccmap(outputnameccfind, 'coord.db',results=ccmapresult,solutions='solp2.txt',image=himage,xcol=3,ycol=4,
           lngcol=1,latcol=2,lngunits='degrees',latunits='degrees',
           interactive='no',update='yes',insystem='fk5',projection='tan',fitgeometry='rxyscale',reject='3')

Refsystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000
Insystem: fk5  Coordinates: equatorial FK5
    Equinox: J2000.000 Epoch: J2000.00000000 MJD: 51544.50000

Coords File: gaiamatchedfindj2000p5  Image: m30.fits
    Database: coord.db  Solution: solp2.txt
Coordinate mapping status
    XI fit ok.  ETA fit ok.
    Ra/Dec or Long/Lat fit rms: 0.0428  0.0657   (arcsec  arcsec)
Coordinate mapping parameters
    Sky projection geometry: tan
    Reference point: 325:05:33.52  -23:10:46.91  (degrees  degrees)
    Reference point: 4956.943  5018.267  (pixels  pixels)
    X and Y scale: 0.039  0.040  (arcsec/pixel  arcsec/pixel)
    X and Y axis rotation: 180.007  0.007  (degrees  degrees)
Wcs mapping status
    Ra/Dec or Long/Lat wcs rms: 0.0428  0.0657   (arcsec  arcsec)
Updating image header wcs

